# Piezo Crystal Voltage Responce

In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mp
import matplotlib.pyplot as plt 
import uncertainties as unc
import scipy as sci

from uncertainties import ufloat
from scipy.optimize import curve_fit

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Import data

In [2]:
# Function Generator Voltage
FV = pd.read_csv('Data/F0000CH1.CSV', header=None, index_col=False, usecols=[3, 4])

# Piezo Voltage
PV = pd.read_csv('Data/F0000CH4.CSV', header=None, index_col=False, usecols=[3, 4])

# Photodetector Voltage 
DV = pd.read_csv('Data/F0000CH2.CSV', header=None, index_col=False, usecols=[3, 4])

In [3]:
# FV.rename(columns={'3':'Time (s)', '4':'Voltage (s)'}, inplace=True)
FV.columns = ['Time (s)', 'Voltage (V)']
PV.columns = ['Time (s)', 'Voltage (mV)']
DV.columns = ['Time (s)', 'Voltage (V)']

## Plot the Data

In [4]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

FV.plot(x = 'Time (s)', y = 'Voltage (V)' , ax = axes[0], legend=False,
        title = 'Voltage output of function generator',
        xlabel = 's',
        ylabel = 'V')

PV.plot(x = 'Time (s)', y = 'Voltage (mV)', ax = axes[1], legend=False,
        title = 'Voltage applied to piezo crystal',
        xlabel = 's',
        ylabel = 'mV')

DV.plot(x = 'Time (s)', y = 'Voltage (V)', ax = axes[2], legend=False,
        title = 'Voltage output of photodetector',
        xlabel = 's',
        ylabel = 'V')

axes[0].grid()
axes[1].grid()
axes[2].grid()
plt.tight_layout()
plt.show()

<Figure size 864x216 with 3 Axes>

## Curve fit the voltage output of photodetector

In [95]:
# Function
def DVFunc(x, a, b, c, d):
    f = a*np.sin(b*x + c) + d
    return f

# Find positions of min and max values of FV
FV_min_pos, FV_max_pos = np.argmin(FV['Voltage (V)']), np.argmax(FV['Voltage (V)'])

# Constrain DV elements to those between min and max of FV
DVTime, DVVolt = DV['Time (s)'][FV_min_pos:FV_max_pos], DV['Voltage (V)'][FV_min_pos:FV_max_pos]

# Curve Fit
popt, pcov = curve_fit(DVFunc, DVTime, DVVolt, 
                       bounds = ([0, 3500, -2*np.pi, -0.4], [0.5, 4500, 2*np.pi, 0.2]))
a, b, c, d = popt

# Plot it out 
ax = plt.figure(figsize = (4, 3))

DV.plot(x = 'Time (s)', y = 'Voltage (V)', legend=False,
        title = 'Voltage output of photodetector',
        xlabel = 's',
        ylabel = 'V',
        label = 'Data')

plt.plot(DVTime, DVFunc(DVTime, *popt), 
         label = r'Curve fit: $y = {0:5.3f} \sin({1:5.3f}\pi x {2:5.3f} \pi) {3:5.3f}$'.format(a, b/np.pi, c/np.pi, d))

plt.ylim(np.min(DV['Voltage (V)'])*1.2)
plt.legend(loc='lower left')
plt.grid()
plt.show()

print(*popt)


<Figure size 288x216 with 0 Axes>

<Figure size 432x288 with 1 Axes>

0.1588063159081293 3971.8554543801342 -0.3097021094298 -0.26870097245881785


## Figuring out the displacement of piezo crystal per voltage